<img src="Slike/vua.png">

U radu ponekad trebamo i podatke iz vanjskih izvora koje koristimo u programu.
Izazov je u tome što nije uvijek moguće pronaći podatke koji su dostupni putem
besplatnih API poziva. U ovakvim slučajevima moramo koristiti *web scraping*.
*Web scraping* je način prikupljanja podataka koji se nalaze na web-stranicama.
To se može učiniti ručno, kopiranjem sadržaja, ili programski (bot). Programski
se podatci mogu prikupiti mnogo brže nego ručno i zato ćemo se usredotočiti na
taj način. Trebamo uzeti u obzir da zakonitost ove prakse nije definirana.
Web-stranice u svojim uvjetima korištenja i u datoteci *robots.txt* obično
opisuju je li to dopušteno, ali imaju problem da kod prepoznavanja. *Web
scraping* programi prikupljaju podatke s web-stranica na isti način kao što bi
to učinio čovjek: otvore web-stranicu i dohvate podatke koje bi inače dohvatio
web-preglednik. Svaka web-stranica ima drugačiju strukturu, zato za svaku moramo
zasebno napraviti program. Web-stranice su kreirane pomoću jezika HTML (engl.
*Hypertext Markup Language*), zajedno s CSS-om (engl. *Cascading Style Sheets*)
i jezikom JavaScript. HTML elementi su odvojeni oznakama i izravno uvode sadržaj
na web-stranicu. Evo kako izgleda osnovni HTML dokument:

<img src="Slike/basic_html_page.png">

Možemo vidjeti da je sadržaj naslova sadržan između oznaka „h1“. Prvi paragraf
nalazi se između oznaka „p“. Na pravoj web-stranici trebamo pronaći oznake s
relevantnim podatcima i iskoristiti ih u svojem programu. Također, moramo
odrediti koje veze (*linkove*) treba istražiti i gdje ih možemo pronaći među
HTML datotekama. Uz sve ove informacije trebali bismo moći prikupiti potrebne
podatke. Koristit ćemo modul *requests* iz prethodne vježbe i modul
*BeautifulSoup*. Modul *requests* će omogućiti slanje HTTP zahtjeva i
prihvaćanje odgovora (<http://docs.python-requests.org/en/master/>). Modul
*BeautifulSoup* se koristiti za analizu HTML datoteka. To je jedan od najčešće
korištenih modula za *web-scarping*. Jednostavan je za korištenje i ima mnogo
značajki koje pomažu pri prikupljanju podataka iz HTML koda
(<https://www.crummy.com/software/BeautifulSoup/bs4/doc/>).

U ovom primjeru pokušat ćemo skupiti podatke s *online* knjižare:
<http://books.toscrape.com/>. Ova web-stranica nije prava knjižara, već je
napravljena za vježbanje skidanja podataka. Cilj nam je prikupiti podatke o
proizvodima na web-stranici (naslov knjige, cijena, dostupnost, slika,
kategorija i ocjena). Prvo upotrijebimo modul *requests* za dohvaćanje HTML koda
glavne stranice i prikažimo dobiveni rezultat.

In [ ]:
import requests
from bs4 import BeautifulSoup

glavni_url = "http://books.toscrape.com/index.html"
odgovor = requests.get(glavni_url)
odgovor.text

Rezultat je prilično neuredan! Učinimo ovo čitljivijim (nastavljamo s kodom dalje bez ponavljanja jer je dio već učitan):

In [ ]:
soup = BeautifulSoup(odgovor.text, 'html.parser')
soup

Definiramo funkciju za preuzimanje i analizu HTML koda jer će nam često trebati tijekom izvođenja programa.

In [ ]:
import requests
from bs4 import BeautifulSoup

def dohvatiStranicu(url):
    odgovor = requests.get(url)
    soup = BeautifulSoup(odgovor.text, 'html.parser')
    return(soup)

Da bismo prikupili podatke o knjizi, moramo pristupiti stranici proizvoda. Prvi
korak sastoji se od pronalaženja poveznica koje prikazuju svaku knjigu. U
pregledniku otvorite glavnu stranicu web-mjesta, kliknite desnom tipkom miša na
naziv proizvoda i kliknite na **inspect**. To će pokazati dio u HTML kodu
web-stranice koji odgovara ovom elementu. Tako smo pronašli prvu vezu na knjigu.

<img src="Slike/inspect.png">

Ovo možete pokušati sa svim drugim knjigama na stranici: struktura je uvijek
ista. Veza proizvoda odgovara atributu „href" oznake „a". Ovo pripada oznaci
„article“ s vrijednošću razreda „product_pod“. To će nam biti izvor za uočavanje
URL-ova proizvoda. *BeautifulSoup* omogućuje da pronađemo one posebne oznake
„article“. Funkciju *find()* možemo koristiti za pronalazak prve pojave ove
oznake u HTML kodu.

In [ ]:
soup = dohvatiStranicu("http://books.toscrape.com/index.html")
soup.find("article", class_ = "product_pod")

Još uvijek imamo previše informacija. Pokušajmo doći do URL-a.

In [ ]:
soup.find("article", class_ = "product_pod").div.a

Puno bolje! Međutim, trebamo samo URL koji se nalazi u vrijednosti „href“.

In [ ]:
soup.find("article", class_ = "product_pod").div.a.get('href')

Uspjeli smo dobiti prvi URL proizvoda s modulom *BeautifulSoup*. Sada možemo iskoristiti funkciju *findAll()* i skupiti sve URL-ove proizvoda na glavnoj web-stranici.

In [ ]:
main_page_products_urls = [x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

main_page_products_urls

Ova funkcija je vrlo korisna za pronalaženje svih vrijednosti odjednom, ali
morate provjeriti jesu li sve prikupljene informacije relevantne. Ponekad jedna
te ista oznaka može sadržavati potpuno različite podatke. Zato je pri odabiru
oznaka važno biti što precizniji. Ovdje smo se odlučili osloniti na oznaku
„article“ s razredom „product_pod“ jer se čini da je to vrlo specifična oznaka i
malo je vjerojatno da ćemo u njoj pronaći podatke koji nisu podatci o proizvodu.
Prethodni URL-ovi odgovaraju njihovoj relativnoj stazi od glavne stranice. Da
bismo ih dovršili, samo im moramo dodati URL glavne stranice:
<http://books.toscrape.com/index.html> (nakon uklanjanja dijela index.html).
Definirajmo novu funkciju za dohvaćanje veza knjiga na bilo kojoj stranici. To
radimo u koracima, tako da provjerimo rezultat nakon svakog koraka.

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    print(soup)

dohvatiURLove('http://books.toscrape.com/index.html')

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    print(url)
   
dohvatiURLove('http://books.toscrape.com/index.html')

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    print(url)
        
dohvatiURLove('http://books.toscrape.com/index.html')

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    print (url)
    
dohvatiURLove('http://books.toscrape.com/index.html')

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    print (vrati)
    
dohvatiURLove('http://books.toscrape.com/index.html')

Na kraju se nalazi funkcija kojoj prosljeđujemo URL stranice i ona pronalazi poveznice na sve proizvode i vraća ih kao listu.

In [ ]:
def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    return (vrati)

Cijeli program trenutno izgleda ovako:

In [ ]:
import requests
from bs4 import BeautifulSoup

glavni_url = "http://books.toscrape.com/index.html"

def dohvatiStranicu(url):
    odgovor = requests.get(url)
    soup = BeautifulSoup(odgovor.text, 'html.parser')
    return(soup)

def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    return (vrati)

Sada pokušajmo dohvatiti URL-ove koji odgovaraju različitim kategorijama proizvoda.

<img src="Slike/inspect2.png">

Pregledom vidimo da oni slijede isti uzorak URL-a: 'catalogue/category/books'.
To možemo iskoristiti da dohvatimo URL-ove kategorija i pokrenuti modul *re*
koji omogućuje da dodamo „catalogue/category/books" kao dio URL-a koji tražimo.

In [ ]:
import re

url_kategorija = [glavni_url + x.get('href') for x in soup.find_all("a", href=re.compile("catalogue/category/books"))]
url_kategorija = url_kategorija[1:] # mičemo prvi zapis jer pokazuje na sve knjiga

url_kategorija

Uspjeli smo dohvatiti URL-ove 50 kategorija! Ne zaboravite uvijek provjeriti što
ste dobili kao rezultat, kako biste bili sigurni da su sve informacije
relevantne. Dobivanje URL-ova može biti korisno ako želimo preuzeti samo
određenu kategoriju knjiga (proizvoda).

Sad kad smo došli do poveznica, prikupimo podatke o knjigama. Znamo kako dobiti
poveznicu na knjige unutar web-stranice. Sad ostaje još izazov kako doći do svih
stranica na kojima su prikazane knjige. Obično se proizvodi prikazuju na više
stranica ili na jednoj stranici, ali kroz pomicanje. Na dnu stranice vidimo da
na konkretnom dućanu imamo 50 stranica i da se nalazimo na prvoj od njih.

<img src="Slike/next.png">

Na sljedećim stranicama se uz gumb *next* nalazi i *previous* za povratak na prethodnu stranicu s proizvodima.

<img src="Slike/previous_next.png">

Kako bismo dohvatili sve URL-ove proizvoda, moramo biti u mogućnosti pregledati sve stranice. To možemo učiniti tako da iterativno pregledamo stranice koje se nalaze na gumbu *next*.

<img src="Slike/next_inspect.png">

Gumb *next* sadrži uzorak „*page*". To možemo upotrijebiti za dohvaćanje URL-ova
sljedećih stranica. Pritom budimo oprezni jer i gumb *previous* ima isti uzorak.
Analizom koda možemo uočiti da u slučajevima u kojima imamo dva uzorka *page*,
trebamo onaj koji se pojavljuje drugi.

In [ ]:
url_stranica = [glavni_url]

dohvatiURL = dohvatiURLove(glavni_url)

soup = dohvatiStranicu(glavni_url)

# kada dobijemo dva podudaranja znači da smo na stranicama koje imaju prethodnu i sljedeću stranicu
# kada imamo samo jedan rezultat nalazimo se na prvoj ili zadnjoj stranici
# trebamo prestati kada dođemo do zadnje stranice

while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(url_stranica) == 1:
    new_url = "/".join(url_stranica[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    url_stranica.append(new_url)
    soup = dohvatiStranicu(new_url)
    
print(url_stranica)

Uspješno smo prikupili URL-ove 50 stranica. Zanimljivo je da je URL tih stranica
vrlo predvidljiv. Mogli smo napraviti ovaj popis tako da povećavamo
„page-X.html“ do 50. Ovo rješenje moglo bi funkcionirati za upravo ovaj primjer,
ali više ne bi funkcioniralo ako se broj stranica promijeni. Jedno rješenje bi
moglo biti povećanje vrijednosti dok ne dođemo na 404. stranicu.

<img src="Slike/404.png">

Ovdje možemo vidjeti da pokušaj odlaska na 51. stranicu ispisuje grešku 404.
Srećom, *requests* zahtjeva ima atribut koji može pokazati status povratka HTML
zahtjeva.

In [ ]:
odgovor = requests.get("http://books.toscrape.com/catalogue/page-50.html")
print("Status kod za stranicu 50: " + str(odgovor.status_code))

odgovor = requests.get("http://books.toscrape.com/catalogue/page-51.html")
print("Status kod za stranicu 51: " + str(odgovor.status_code))

U sljedećem koraku moramo dohvatiti URL-ove svih proizvoda na svim stranicama.
Ovaj korak je vrlo jednostavan jer već imamo popis svih stranica s proizvodima i
funkciju za dohvaćanje URL-ova proizvoda. Prelistajmo sve stranice i primijenimo
svoju funkciju:

In [ ]:
import requests, re
from bs4 import BeautifulSoup

glavni_url = "http://books.toscrape.com/index.html"

def dohvatiStranicu(url):
    odgovor = requests.get(url)
    soup = BeautifulSoup(odgovor.text, 'html.parser')
    return(soup)

def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    return (vrati)

def dohvatiSveStranice(url):
    url_stranica = [url]
    dohvatiURL = dohvatiURLove(glavni_url)
    soup = dohvatiStranicu(glavni_url)
    # kada dobijemo dva podudaranja znači da smo na stranicama koje imaju prethodnu i sljedeću stranicu
    # kada imamo samo jedan rezultat nalazimo se na prvoj ili zadnjoj stranici
    # trebamo prestati kada dođemo do zadnje stranice
    while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(url_stranica) == 1:
        new_url = "/".join(url_stranica[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
        url_stranica.append(new_url)
        soup = dohvatiStranicu(new_url)
    return (url_stranica)

sve_stranice = dohvatiSveStranice(glavni_url)

sve_knjige = []
for stranica in sve_stranice:
    sve_knjige.extend(dohvatiURLove(stranica))
    
print(len(sve_knjige))

Napokon smo dobili 1000 URL-ovaza sve knjige koje se nalaze na web-mjestu.
Stavimo to u funkciju i počnimo prikupljati podatke o pojedinim knjigama.

In [ ]:
import requests, re
from bs4 import BeautifulSoup

glavni_url = "http://books.toscrape.com/index.html"

def dohvatiStranicu(url):
    odgovor = requests.get(url)
    soup = BeautifulSoup(odgovor.text, 'html.parser')
    return(soup)

def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    return (vrati)

def dohvatiSveStranice(url):
    url_stranica = [url]
    dohvatiURL = dohvatiURLove(glavni_url)
    soup = dohvatiStranicu(glavni_url)
    # kada dobijemo dva podudaranja znači da smo na stranicama koje imaju prethodnu i sljedeću stranicu
    # kada imamo samo jedan rezultat nalazimo se na prvoj ili zadnjoj stranici
    # trebamo prestati kada dođemo do zadnje stranice
    while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(url_stranica) == 1:
        new_url = "/".join(url_stranica[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
        url_stranica.append(new_url)
        soup = dohvatiStranicu(new_url)
    return (url_stranica)

def dohvatiSveLinkoveKnjiga (sve_stranice):
    sve_knjige = []
    for stranica in sve_stranice:
        sve_knjige.extend(dohvatiURLove(stranica))
    return (sve_knjige)

sve_stranice = dohvatiSveStranice(glavni_url)

sve_knjige = dohvatiSveLinkoveKnjiga(sve_stranice)

   
print(len(sve_knjige))

Posljednji korak sastoji se od skupljanja podataka za svaku pojedinu knjigu.
Istražimo najprije kako su informacije strukturirane na stranicama proizvoda.

<img src="Slike/product_inspect.png">

Skupimo sad sve podatke. Kako će ovo malo potrajati, ubacit ćemo i dio koji
ispisuje vrijeme izvođenja.

In [ ]:
%%time
#kreiramo prazne liste za prikupljanje podataka
naziv = []
cijena = []
zaliha = []
url_slike = []
kategorija = []
ocjena = []
# skidanje podataka o svim knjigama - traje malo duže jer treba posjetiti 1000 stranica
for url in sve_knjige:
    soup = dohvatiStranicu(url)
    # naivi
    naziv.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    # cijena
    cijena.append(soup.find("p", class_ = "price_color").text[2:]) # makni oznaku valute
    # zaliha
    zaliha.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # makni tekst
    # link na sliku
    url_slike.append(url.replace("index.html", "") + soup.find("img").get("src"))
    # kategorija
    kategorija.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])
    # ocjena
    ocjena.append(soup.find("p", class_ = re.compile("star-rating")).get("class")[1])

Na kraju to spremimo u novu funkciju te napravimo izlaz iz funkcije koja će
vratiti tablicu sa svim podatcima korištenjem modula *pandas*.

In [ ]:
import requests, re, pandas
from bs4 import BeautifulSoup

def dohvatiStranicu(url):
    odgovor = requests.get(url)
    soup = BeautifulSoup(odgovor.text, 'html.parser')
    return(soup)

def dohvatiURLove(url):
    soup = dohvatiStranicu(url)
    #pretvorimo url u listu s elementima odvojenim znakom '/'
    url = (url.split("/"))
    #maknimo iz liste zadnji element (index.html)
    url = url[:-1]
    #povežimo nazad u string
    url = '/'.join(url)
    #kreiramo listu s cjelim URL-ovima
    vrati = [url + '/' + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]
    return (vrati)

def dohvatiSveStranice(url):
    url_stranica = [url]
    dohvatiURL = dohvatiURLove(glavni_url)
    soup = dohvatiStranicu(glavni_url)
    # kada dobijemo dva podudaranja znači da smo na stranicama koje imaju prethodnu i sljedeću stranicu
    # kada imamo samo jedan rezultat nalazimo se na prvoj ili zadnjoj stranici
    # trebamo prestati kada dođemo do zadnje stranice
    while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(url_stranica) == 1:
        new_url = "/".join(url_stranica[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
        url_stranica.append(new_url)
        soup = dohvatiStranicu(new_url)
    return (url_stranica)

def dohvatiSveLinkoveKnjiga (glavni_url):
    sve_stranice = dohvatiSveStranice(glavni_url)
    sve_knjige = []
    for stranica in sve_stranice:
        sve_knjige.extend(dohvatiURLove(stranica))
    return (sve_knjige)

def dohvatiSvePodatke(glavni_url):
    sve_knjige = dohvatiSveLinkoveKnjiga (glavni_url)
    naziv = []
    cijena = []
    zaliha = []
    url_slike = []
    kategorija = []
    ocjena = []
    # skidanje podataka o svim knjigama - traje malo duže jer treba posjetiti 1000 stranica
    for url in sve_knjige:
        soup = dohvatiStranicu(url)
        # naivi
        naziv.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
        # cijena
        cijena.append(soup.find("p", class_ = "price_color").text[2:]) # get rid of the pound sign
        # zaliha
        zaliha.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # get rid of non numerical characters
        # link na sliku
        url_slike.append(url.replace("index.html", "") + soup.find("img").get("src"))
        # kategorija
        kategorija.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])
        # ocjena
        ocjena.append(soup.find("p", class_ = re.compile("star-rating")).get("class")[1])
    
    preuzeti_podaci = pandas.DataFrame({'naziv': naziv, 
                                    'cijena': cijena, 
                                    'zaliha': zaliha, 
                                    "url_slike": url_slike, 
                                    "kategorija": kategorija, 
                                    "ocjena": ocjena})
    return(preuzeti_podaci)
    

glavni_url = "http://books.toscrape.com/index.html"
podaci = dohvatiSvePodatke(glavni_url)
podaci


U tablici sad imamo 1000 redaka i 6 stupaca.  
**Ovim smo završili naš seminar iz uvoda u Python.**   
Ostaje vam još zadnji zadatak.

<br><div class="alert alert-info"><b>Vježba</b></div>

Napišite program koji će u obliku tablice vratiti podatke o temperaturi mora.  
Web-stranica se nalazi na poveznici https://meteo.hr/podaci.php?section=podaci_vrijeme&param=more_n.  
U radu na zadatku u prvoj ćeliji riješite dio koji se tiče učitavanja HTML koda sa stranice, pa nastavite u idućoj ćeliji raditi na programu.


In [ ]:
# učitavanje HTML-a


In [ ]:
# obrada preuzetog koda


In [ ]:
# vizualizacija podataka


<br><div class="alert alert-info"><b>Kraj</b></div></br>